In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Tuple
from ca2_analysis import ca2Data, peakAndTroughIndices, ca2Metrics
%matplotlib inline
pd.set_option("display.precision", 2)
pd.set_option("display.expand_frame_repr", False)

In [ ]:
def contentsOfDir(dir_path: str, search_terms: List[str]) -> Tuple[List[str], List[Tuple[str]]]:
  if os.path.isdir(dir_path):
    base_dir = dir_path
    for search_term in search_terms:
      glob_search_term = '*' + search_term + '*'
      file_paths = glob.glob(os.path.join(dir_path, glob_search_term))
      if len(file_paths) > 0:
        break
  else:
    # presume it's actually a single file path
    base_dir = os.path.dirname(dir_path)
    file_paths = [dir_path]
  files = []
  for file_path in file_paths:
      file_name, file_extension = os.path.splitext(os.path.basename(file_path))
      files.append((file_name, file_extension))
  return base_dir, files

In [ ]:
def plotCa2Signals(
    time_stamps: np.ndarray,
    signal: np.ndarray,
    peak_indices: np.ndarray,
    trough_indices: np.ndarray,
    plot_title: str='',
    plot_smoothed_signal: bool=True
):
    plt.title(plot_title)
    if plot_smoothed_signal:
        plt.plot(time_stamps, signal)
    else:
        plt.scatter(time_stamps, signal, s=2, facecolors='none', edgecolors='g')    
    plt.scatter(time_stamps[peak_indices], signal[peak_indices], s=80, facecolors='none', edgecolors='b')
    plt.scatter(time_stamps[trough_indices], signal[trough_indices], s=80, facecolors='none', edgecolors='r')
    plt.show()

In [ ]:
def runTestSuite():
    test_dir, test_files = contentsOfDir(dir_path='../test_data', search_terms=['.xlsx'])
    for file_name, file_extension in test_files:
        time_stamps, input_signal = ca2Data(os.path.join(test_dir, file_name + file_extension))

        peak_indices, trough_indices = peakAndTroughIndices(
            input_signal,
            time_stamps,
            expected_frequency_hz=1.0
        )

        # time_stamps_of_troughs = time_stamps[trough_indices]
        # print(f'trough times: {time_stamps_of_troughs}')
        # time_stamps_of_peaks = time_stamps[peak_indices]
        # print(f'peak times: {time_stamps_of_peaks}')
# trough times: [        1.385  2.379  3.29   4.383  5.349  6.232  7.24   8.249  9.384 10.378 11.387 12.382 13.376]
# peak times:   [ 0.418  1.427  2.421  3.416  4.425  5.419  6.428  7.423  8.417  9.426 10.42  11.429 12.424 13.418]

        # plotCa2Signals(
        #     time_stamps, 
        #     input_signal,
        #     peak_indices,
        #     trough_indices,
        #     file_name
        # )

        all_metrics = ca2Metrics(
            input_signal,
            time_stamps,
            expected_frequency_hz=1.0
        )
        print()
        print(f'metrics for {file_name}')
        for metrics in all_metrics:
            p2p_order = metrics['p2p_order']
            average_metrics = metrics['mean_metric_data']
            print(f'{p2p_order} average metrics: {average_metrics}')
        print()

In [ ]:
runTestSuite()